In [7]:
import os
import sys
import numpy as np
import cv2
import caffe
import dlib
import matplotlib.pyplot as plt
from math import sqrt
import cv2
from sklearn import *
from numpy import *
import glob

In [8]:
caffe_root='/home/gtx1080ti/Documents/hanyou2/gazecapture/'
#sys.path.insert(0, caffe_root + 'python')

if os.path.isfile(caffe_root + 'snapshots/itracker25x_iter_92000.caffemodel'):
    print('CaffeNet found.')
else:
    print('Downloading pre-trained CaffeNet model...')
    #!../scripts/download_model_binary.py ../models/bvlc_reference_caffenet


CaffeNet found.


In [9]:
#specify the path of dataset
name='0206002' 
home='/home/gtx1080ti/Documents/hanyou2/0206/'
path='/home/gtx1080ti/Documents/hanyou2/0206/'+name+'/'

In [10]:
def getError(truth, noise):
        mean_error=[]
        for i in range(len(truth)):
            error=sqrt((noise[i][0]-int(truth[i][0]))**2+
                               (noise[i][1]-int(truth[i][1]))**2)
            mean_error.append(error)
        error_d=sum(mean_error) / float(len(mean_error))
        return error_d

In [34]:
def getData(name):
    import csv

    tsv_data=home+'data/'+name+'f.tsv'

    #Load the data from tobii
    tobii_all_data=[]
    with open(tsv_data) as tf:
        treader=csv.reader(tf, delimiter='\t')
        for line in treader:
            tobii_all_data.append((line[0],line[8],line[9],line[10],line[11],line[12],line[13]))
    #The fps of Tobii is 60, so only use half of the data

    tobii_data=[]
    for i in range(1,len(tobii_all_data),2):
        tobii_data.append(tobii_all_data[i])
    import pickle
    low_data= pickle.load( open( home+name+'/low_data_cut', "rb" ) )
    high_data= pickle.load( open( home+name+'/high_data_cut', "rb" ) )
    print(len(low_data))

    #get the original accuracy
    #store valid inputs for cnn in valid_high
    low_coor=[]
    high_coor=[]
    valid_high=[] #Only keep the points when using the saliency images on the screen
    for i in range(len(high_data)):
        if high_data[i][1]!='' and high_data[i][1]!='phase1.jpg' and high_data[i][1]!='phase2.jpg' and high_data[i][1]!='fixation.png':
            valid_high.append(high_data[i])
            low_coor.append((low_data[i][1],low_data[i][2]))
            high_coor.append((high_data[i][2],high_data[i][3]))
    print(getError(low_coor,high_coor))
    print(len(valid_high))
    print()
    return (valid_high,low_coor,high_coor)

(valid_high_5, low_d_5,high_d_5)=getData('0206005')
(valid_high_2, low_d_2,high_d_2)=getData('0206002')


5954
161.97653081525846
4409

6911
135.9990608392908
4496



When the inputs for the regression contain a series of continuous eye gazes, we need to compute the continuity list

In [35]:
continuity=[0] #use this list to record the length of continuous sequence for regression.
x=0
for i in range(1,len(valid_high)):
    if valid_high[i][0]-valid_high[i-1][0]==1:
        x=x+1
    else:
        x=0
    continuity.append(x)
cnn_inputs=[] #will be used for the cnn regression model, the img index in terms of Tobii
for i in range(len(continuity)-3):
    if continuity[i]>=2:
        cnn_inputs.append(valid_high[i])
len(cnn_inputs)
valid_high=cnn_inputs

'continuity=[0] #use this list to record the length of continuous sequence for regression.\nx=0\nfor i in range(1,len(valid_high)):\n    if valid_high[i][0]-valid_high[i-1][0]==1:\n        x=x+1\n    else:\n        x=0\n    continuity.append(x)\ncnn_inputs=[] #will be used for the cnn regression model, the img index in terms of Tobii\nfor i in range(len(continuity)-3):\n    if continuity[i]>=2:\n        cnn_inputs.append(valid_high[i])\nlen(cnn_inputs)\nvalid_high=cnn_inputs'

In [36]:
#def getTrainTEst(valid_high, name,low_coor,high_coor):

def getTrainTEst(valid_high, name,low_coor,high_coor):
    #path tp all the saliency maps of the screenshot
    saliency_path="/home/gtx1080ti/Documents/hanyou2/BenchmarkIMAGES/saliency_full/" 
    #path to the images that respresents each eye gazes
    fixation_path=home+name+"/calibration/gazes/"
    train_saliency=[]
    train_Y=[]
    train_g=[]

    l_te=0
    h_te=900
    l_tr=900
    h_tr=len(valid_high)

    for i in valid_high[l_tr:h_tr]:
        img=cv2.imread(saliency_path+i[1])
        img=np.einsum('lij->jil',np.array(img))
        img=[np.einsum('ij->ji',img[0])]
        img=np.einsum('lij->ijl',np.array(img))
        train_saliency.append(img)

        train_Y.append((i[2],i[3]))

        #start do multiple channels
        img=cv2.imread(fixation_path+str(i[0])+".jpg")
        img=np.einsum('lij->jil',np.array(img))
        img=[np.einsum('ij->ji',img[0])]
        img=np.einsum('lij->ijl',np.array(img))
        #start do multiple channels
        train_g.append(img)



    test_saliency=[]
    test_Y=[]
    test_g=[]

    for i in valid_high[l_te:h_te]:
        img=cv2.imread(saliency_path+i[1])

        img=np.einsum('lij->jil',np.array(img))
        img=[np.einsum('ij->ji',img[0])]
        img=np.einsum('lij->ijl',np.array(img))

        test_saliency.append(img)

        test_Y.append((i[2],i[3]))

        img=cv2.imread(fixation_path+str(i[0])+".jpg")
        img=np.einsum('lij->jil',np.array(img))
        img=[np.einsum('ij->ji',img[0])]
        img=np.einsum('lij->ijl',np.array(img))

        test_g.append(img)
        
        
    train_saliency=np.array(train_saliency)
    train_Y=np.array(train_Y)
    train_g=np.array(train_g)
    test_saliency=np.array(test_saliency)
    test_Y=np.array(test_Y)
    test_g=np.array(test_g)
    print(train_saliency.shape)
    print(train_g.shape)
    print(test_saliency.shape)
    print(test_g.shape)

    print(getError(low_coor,high_coor))
    
    train_x=np.concatenate((train_g,train_saliency),axis=3)
    test_x=np.concatenate((test_g,test_saliency),axis=3)
    print(train_x.shape)
    del train_g,train_saliency,test_g,test_saliency
    print()
    return (train_x,train_Y,test_x,test_Y)



In [37]:
# Produce training data, here I just use the current eye gaze instead of the recent several gazes, 
# because this tends to produce a better result
(trainx1,trainy1,testx1,testy1)=getTrainTEst(valid_high_2,'0206002',low_d_2,high_d_2)

(trainx2,trainy2,testx2,testy2)=getTrainTEst(valid_high_5,'0206005',low_d_5,high_d_5)
del valid_high_2, low_d_2,high_d_2
del valid_high_5, low_d_5,high_d_5
trainx=np.concatenate((trainx1, testx1),axis=0)
testx=np.concatenate((trainx2,testx2),axis=0)
trainy=np.concatenate((trainy1,testy1),axis=0)
testy=np.concatenate((trainy2,testy2),axis=0)
print(trainx.shape)
print(trainy.shape)
print(testx.shape)
print(testy.shape)

del trainx2,trainx1,trainy1,trainy2
del testx1,testx2,testy1,testy2


(3596, 512, 640, 1)
(3596, 512, 640, 1)
(900, 512, 640, 1)
(900, 512, 640, 1)
135.9990608392908
(3596, 512, 640, 2)

(3509, 512, 640, 1)
(3509, 512, 640, 1)
(900, 512, 640, 1)
(900, 512, 640, 1)
161.97653081525846
(3509, 512, 640, 2)

(4496, 512, 640, 2)
(4496, 2)
(4409, 512, 640, 2)
(4409, 2)


In [38]:
# You may want to store the training and testing data for using them later
'''
import pickle
trainx= pickle.load(open( home+name+'/trainx', "rb" ) )#raw error=(159.506+172.250)/2=166
trainy=pickle.load(open( home+name+'/trainy', "rb" ) )
testx= pickle.load(open( home+name+'/testx', "rb" ) )
testy=pickle.load(open( home+name+'/testy', "rb" ) )
#import pickle

#pickle.dump(trainx, open( home+name+'/trainx', "wb" ) )
#pickle.dump(trainy, open( home+name+'/trainy', "wb" ) )
#pickle.dump(testx, open( home+name+'/testx', "wb" ) )
#pickle.dump(testy, open( home+name+'/testy', "wb" ) )'''

'\nimport pickle\ntrainx= pickle.load(open( home+name+\'/trainx\', "rb" ) )#raw error=(159.506+172.250)/2=166\ntrainy=pickle.load(open( home+name+\'/trainy\', "rb" ) )\ntestx= pickle.load(open( home+name+\'/testx\', "rb" ) )\ntesty=pickle.load(open( home+name+\'/testy\', "rb" ) )\n#import pickle\n\n#pickle.dump(trainx, open( home+name+\'/trainx\', "wb" ) )\n#pickle.dump(trainy, open( home+name+\'/trainy\', "wb" ) )\n#pickle.dump(testx, open( home+name+\'/testx\', "wb" ) )\n#pickle.dump(testy, open( home+name+\'/testy\', "wb" ) )'

In [61]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Conv2D, Flatten, Input, normalization,MaxPooling2D,concatenate
 import keras
import tensorflow
import struct

In [68]:
from random import shuffle
ind_list=[i for i in range(len(trainx))]
shuffle(ind_list)
trainx=trainx[ind_list,:,:,:]
trainy=trainy[ind_list,]

In [69]:
input_s=Input(shape=(512,640,2))

conv1_s=Conv2D(12,(9,9),strides=(3,3),activation='relu',data_format = 'channels_last')(input_s)

pool1_s=MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format = 'channels_last')(conv1_s)

conv2_s=Conv2D(48,(5,5),activation='relu',data_format = 'channels_last')(pool1_s)

pool2_s=MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format = 'channels_last')(conv2_s)

conv3_s=Conv2D(12,(2,2),padding='same',activation='relu',data_format = 'channels_last')(pool2_s)

pool3_s=MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format = 'channels_last')(conv3_s)

#norm1_s=keras.layers.BatchNormalization(axis=2, momentum=0.99, epsilon=0.001, center=True, scale=True)(pool1_s)

flat1_s=Flatten()(pool3_s)

dense1_s=Dense(units=64,activation='relu')(flat1_s)


dense2_s=Dense(units=16,activation='relu')(dense1_s)


dense2=Dense(units=2)(dense2_s)


model=Model(inputs=[input_s],output=[dense2])


/home/gtx1080ti/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


In [70]:
model.compile(loss='mean_squared_error',
        optimizer=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=0.0001, decay=0.000), 
        #optimizer=keras.optimizers.SGD(lr=0.003, momentum=0.9, decay=0.00001, nesterov=True),
        metrics=['accuracy'])


In [71]:
for i in range(5):
    history = model.fit([trainx], [trainy], epochs=1, batch_size=64, 
             verbose=True)

    predY=model.predict([testx])
    tmp=getError(predY,testy)
    print(tmp)

    predY=model.predict([trainx])
    tmp=getError(predY,trainy)
    print(tmp)

Epoch 1/1
4496/4496 [==============================] - 8s 2ms/step - loss: 143308.7818 - acc: 0.6428
367.6597982675373
298.5920264718227
Epoch 1/1
4496/4496 [==============================] - 8s 2ms/step - loss: 25037.0547 - acc: 0.7983
168.05233157609052
121.81855451375833
Epoch 1/1
4496/4496 [==============================] - 8s 2ms/step - loss: 10194.3059 - acc: 0.8921
150.74889021956878
97.71949965106819
Epoch 1/1
4496/4496 [==============================] - 8s 2ms/step - loss: 8533.7397 - acc: 0.8939
144.290004543273
90.6017250014552
Epoch 1/1
4496/4496 [==============================] - 8s 2ms/step - loss: 7837.7192 - acc: 0.8977
141.52756386111
86.94261149471463


In [131]:
import pickle
#model.save(home+name+'/cnn.h5')